In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#scraping data
stats = pd.read_html('https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
shooting = pd.read_html('https://fbref.com/en/comps/Big5/shooting/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
sca = pd.read_html('https://fbref.com/en/comps/Big5/gca/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
passing = pd.read_html('https://fbref.com/en/comps/Big5/passing/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
passing_types = pd.read_html('https://fbref.com/en/comps/Big5/passing_types/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
defense = pd.read_html('https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
posession = pd.read_html('https://fbref.com/en/comps/Big5/possession/players/Big-5-European-Leagues-Stats', header=[0,1])[0]
misc = pd.read_html('https://fbref.com/en/comps/Big5/misc/players/Big-5-European-Leagues-Stats', header=[0,1])[0]

frames = [stats,shooting, sca, passing, passing_types, defense, posession, misc]

#collapsing multilevel headers
for frame in frames:
    frame.columns = frame.columns.to_series().apply(' '.join)
    
#removing unnecessary rows
for frame in frames:
    frame.drop(frame[frame['Unnamed: 1_level_0 Player']=='Player'].index, inplace=True)
    
#merging dataframes    
merged_df = stats.merge(shooting.merge(sca.merge(passing.merge(passing_types.merge(defense.merge(posession.merge(misc)))))))

#renaming columns
merged_df['Player'] = merged_df['Unnamed: 1_level_0 Player']
merged_df['Nation'] = merged_df['Unnamed: 2_level_0 Nation']
merged_df['Position'] = merged_df['Unnamed: 3_level_0 Pos']
merged_df['Squad'] = merged_df['Unnamed: 4_level_0 Squad']
merged_df['League'] = merged_df['Unnamed: 5_level_0 Comp']
merged_df['Age'] = merged_df['Unnamed: 6_level_0 Age']
merged_df['Matches Played'] = merged_df['Playing Time MP']
merged_df['Minutes Played'] = merged_df['Playing Time Min']
merged_df['Goals'] = merged_df['Performance Gls']
merged_df['Assists'] = merged_df['Performance Ast']
merged_df['Goals per90'] = merged_df['Per 90 Minutes Gls']
merged_df['Assists per90'] = merged_df['Per 90 Minutes Ast']
merged_df['xG'] = merged_df['Expected xG']
merged_df['npxG'] = merged_df['Expected npxG']
merged_df['xA'] = merged_df['Expected xA']
merged_df['xG per 90'] = merged_df['Per 90 Minutes xG']
merged_df['npxG per 90'] = merged_df['Per 90 Minutes npxG']
merged_df['xA per 90'] = merged_df['Per 90 Minutes xA']
merged_df['Shots'] = merged_df['Standard Sh']
merged_df['Shots on target'] = merged_df['Standard SoT']
merged_df['Shots per 90'] = merged_df['Standard Sh/90']
merged_df['Shots on target per 90'] = merged_df['Standard SoT/90']
merged_df['SCA'] = merged_df['SCA SCA']
merged_df['SCA per 90'] = merged_df['SCA SCA90']
merged_df['GCA'] = merged_df['GCA GCA']
merged_df['GCA per 90'] = merged_df['GCA GCA90']
merged_df['Passes attempted'] = merged_df['Total Att']
merged_df['Passing accuracy'] = merged_df['Total Cmp%']
merged_df['Key Passes'] = merged_df['Unnamed: 26_level_0 KP']
merged_df['Final third passes'] = merged_df['Unnamed: 27_level_0 1/3']
merged_df['PPA'] = merged_df['Unnamed: 28_level_0 PPA']
merged_df['Progressive passes'] = merged_df['Unnamed: 30_level_0 Prog']
merged_df['Live-ball passes'] = merged_df['Pass Types Live']
merged_df['Dead-ball passes'] = merged_df['Pass Types Dead']
merged_df['Tackles'] = merged_df['Tackles Tkl']
merged_df['Pressures attempted'] = merged_df['Pressures Press']
merged_df['Blocks'] = merged_df['Blocks Blocks']
merged_df['Interceptions'] = merged_df['Unnamed: 28_level_0 Int']
merged_df['Touches'] = merged_df['Touches Touches']
merged_df['Dribbles attempted'] = merged_df['Dribbles Att']
merged_df['Dribbles %'] = merged_df['Dribbles Succ%']
merged_df['Carries'] = merged_df['Carries Carries']
merged_df['Fouls'] = merged_df['Performance Fls']

#selecting important columns
final_df = merged_df[['Player','Nation','Position','Squad','League','Age','Matches Played','Minutes Played','Goals','Assists',
                      'Goals per90','Assists per90','xG','npxG','xA','xG per 90','npxG per 90','xA per 90','Shots',
                      'Shots on target','Shots per 90','Shots on target per 90','SCA','SCA per 90','GCA','GCA per 90',
                     'Passes attempted','Passing accuracy','Key Passes','Final third passes','PPA','Progressive passes',
                     'Live-ball passes','Dead-ball passes','Tackles','Pressures attempted','Blocks','Interceptions',
                     'Touches','Dribbles attempted','Dribbles %','Carries','Fouls']]

#clearing problematic columns
final_df['Nation'] = [i[3:] for i in final_df['Nation']]
final_df['League'] = [i[3:] for i in final_df['League']]
final_df['Age'] = [i[:2] for i in final_df['Age']]

#filling nan values
final_df.fillna(0, inplace=True)

#converting stats to required format
def float_converter(cols):
    for c in cols:
        final_df[c] = final_df[c].astype(str)
        new_data = []
        for val in final_df[c]:
            val = val.replace(',','.')
            val = float(val)
            #val = val.replace('.',',')
            new_data.append(val)

        final_df[c] = new_data
        
floats = final_df[['Goals per90','Assists per90', 'xG', 'npxG', 'xA', 'xG per 90', 'npxG per 90',
       'xA per 90','Shots per 90','Shots on target per 90', 'SCA per 90','GCA per 90','Passing accuracy','Dribbles %']]
float_converter(floats)


def int_converter(cols):
    for c in cols:
        final_df[c] = final_df[c].astype(str)
        new_data = []
        for val in final_df[c]:
            val = int(val)
            new_data.append(val)

        final_df[c] = new_data
        
ints = final_df[['Age','Matches Played', 'Minutes Played', 'Goals', 'Assists','Shots', 'Shots on target','SCA','GCA','Passes attempted',
                'Key Passes','Final third passes', 'PPA', 'Progressive passes', 'Live-ball passes',
               'Dead-ball passes', 'Tackles', 'Pressures attempted', 'Blocks',
               'Interceptions', 'Touches', 'Dribbles attempted','Carries', 'Fouls']]
int_converter(ints)

#removing unnecessary spaces from Nation and Leaegue columns
final_df['Nation'] = final_df['Nation'].str.strip()
final_df['League'] = final_df['League'].str.strip()

#exporting data
final_df.to_excel('fbrefdata.xlsx', index=False)